# Witmielie Bemarking

## Berekening van 'n *benchmark* prys

### Wit Mielies onder besproeiing - 2016/2017

**Area:** 14ha

**SAFEX Kontrakmaande:** Feb, Mar, Apr, Mei, Jul, Sep, Des

**Bemarkingstrategie:**




**Inset Veranderlikes:**

- Kp - produksiekoste
- Pb - benchmark prys
- Rk - Opbrengskoers (opbrengs op kapitaal)
- Y - Opbrengs per hektaar
- Y - Oesskatting (totale oes op alle hektare)
- pk - Persentasie van oes wat produksiekoste dek 
- pn - Gedeelte van pk wat voor die oes verhandel word
- n - aantal verhandelingspakkies (maks n word bepaal deur minimum kontrakgrootte) 
- t0, t1, …, tn - tydsberekening van verkope
- Ks - stoorkoste/ton
- Ko - opsieprys

**Uitsette**

- Verspreiding van totale wins
- Gelykbreekpunt (persentiel van winsverspreiding)

**Verlangde Insig:**
    
- Effek van pk, pn, n op totale wins
- Effek van tydsberekening op totale wins

In [2]:
import numpy as np
import pandas as pd

from preprocessing import resample

import datetime as dt
import quandl as q
q.ApiConfig.api_key = 'MBaawgy9DEZJHU3NkcNd'

import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode()

In [3]:
Kp = 255456 # Total production cost in Rand (sluit versekering uit)
ha = 14 # Number of hectares
Yha = 13 # ton / ha yield
Y = Yha*ha # Yield estimate for total hectares planted
MARR = 0.4 # Minimum acceptable return
Pb = 3700 # Benchmark price estimate (can also be a PDF)

In [4]:
R = Kp * (1 + MARR)
Rha = R / ha
Rton = R / Y

The total production cost is not known beforehand but can be estimated with great accuracy.  The benchmark price should be based on the market outlook for the season ahead, the total production cost, the estimated yield and reasonable minimum acceptable rate of return (MARR).


In [5]:
def hist(d):
    py.offline.iplot([go.Histogram(x=d)])
    
def line(d):
    trace = go.Scatter(x = d.index, y=d)
    py.offline.iplot([trace])

## The tale of two distributions ##

The input price distribution 

In [6]:
wmazfeb17 = q.get("SAFEX/WMAZG2017")
wmazmar17 = q.get("SAFEX/WMAZH2017")
wmazapr17 = q.get("SAFEX/WMAZJ2017")
wmazmay17 = q.get("SAFEX/WMAZK2017")
wmazjul17 = q.get("SAFEX/WMAZN2017")
wmazsep17 = q.get("SAFEX/WMAZU2017")
wmazdec17 = q.get("SAFEX/WMAZZ2017")

In [7]:
wmazdec17.ix[wmazdec17.index.max()]

Open                        NaN
First Trade Price        2240.0
Low                      2230.0
High                     2252.8
Close                       NaN
Change                      NaN
Volume                    473.0
Open Interest             834.0
Vol                         NaN
Contract Size               NaN
Open Interest in Rand       NaN
Name: 2017-02-13 00:00:00, dtype: float64

In [8]:
daily_price = resmaple(wmazmar17)
hist(daily_price)

In [9]:
daily_return = np.log(daily_price).diff()
days = len(daily_price)
mu = daily_return.mean()
sigma = daily_return.std()

In [10]:
line(daily_price)

## *Random Walk with drift* model

*Drift* gelyk aan historiese skatting van gemiddelde daaglikse opbrengs

Standaard afwyking van *drift* gelyk aan skatting van historiese daaglikse opbrengs

**Idees:**

- Voeg 'n deterministiese tendens by volgens markuitkyk asook *futures* pryse in verder leweringsmaande
- Neem seisoenale effekte ook in ag.


In [11]:
# Nuutste prys
price_last = daily_price[daily_price.index.max()]

# Deterministiese tendens

# Generate 10 000 random returns with a normal distribution derived from historical statistics
ret_rand = (np.random.randn(10000) + mu) * sigma

In [12]:
hist(ret_rand)

In [13]:
# Create a random walk 100 days into the future
scenarios = 10000
days = 100
p = np.zeros((days, scenarios))
p[0,:] = price_last

drift = lambda: (np.random.randn() + mu) * sigma
for i in range(0, scenarios):
    for t in range(1, days):
        p[t,i] = p[t-1,i]*np.exp(drift())


### Verspreiding van gemiddelde verwagte prys vir die volgende 100 dae

In [14]:
hist(np.mean(p, axis=0))

### Simuleer 'n verhandelingstrategie wat *randomly* oor die volgende 100 verkoop:

In [15]:
num_trades = 10 # gemiddelde
# list of tuples (scenario, day, price)
trades = list()

coin = lambda p: np.random.randint(0,100) < num_trades
kosteplus = lambda price: price > Rton 
min500wins = lambda price: price - Rton > 500

def trade(strategie, price):
    return strategie(price)

strategie = lambda p: min500wins(p) and coin(p)

for i in range(0, len(p[0,:])):
    for t in range(1, len(p[:,0])):
        if (trade(strategie, p[t,i])):
            trades.append((i, t, p[t,i]))

In [16]:
len(trades)

87375

 ### Verspreiding van gemiddelde prys behaal deur strategie

In [17]:
df_t = pd.DataFrame(trades, columns=["Scenario", "Day", "Price"])
vwap = df_t.groupby(["Scenario"])["Price"].mean()

In [18]:
hist(vwap) 
# behoort normaal versprei te wees weens die sentraal limiet stelling v statistiek

### t-statistiek

In [19]:
x0 = np.mean(np.mean(p, axis=0))

In [20]:
x = vwap.mean()

In [21]:
s = vwap.std()

In [22]:
n = vwap.count()
n

9947

In [23]:
t=(x-x0)/(s/np.sqrt(n))
t

18.470760143986244

## Wins verspreiding

In [24]:
profit = vwap - Rton
hist(profit) # R / ton wins

In [25]:
profit.describe()

count    9947.000000
mean      909.612563
std       231.285277
min       502.350047
25%       734.079454
50%       862.832665
75%      1040.575234
max      2398.851005
Name: Price, dtype: float64

In [26]:
# Totale wins
total_profit = profit*Y
hist(total_profit)

** Die waarskynlikheid dat die strategie haalbaar is gegewe die 100 dae prysingsmodel **

In [27]:
(profit.count() / scenarios)*100

99.469999999999999